In [ ]:
import ipywidgets as widgets
from IPython.display import display

widget = widgets.Text(
    placeholder='Type in your API key here',
    description='API Key:',
    disabled=False   
)

display(widget)

In [ ]:
# Create LLM engine

from smolagents import LiteLLMModel
from typing import Dict

base_model = LiteLLMModel(model_id="o3-mini-2025-01-31", api_key=widget.value)

Message = Dict[str, str]

In [ ]:
# Import agent

# from causal_world_modelling_agent.agents.managers import ManagerAgentFactory
# from causal_world_modelling_agent.agents.causal_discovery.atomic.atomic_discovery_agent import AtomicDiscoveryAgentFactory
# from causal_world_modelling_agent.agents.causal_discovery.atomic.self_iterative_agent import SelfIterativeDiscoveryAgentFactory
from causal_world_modelling_agent.agents.causal_discovery.atomic.atomic_rag_agent import AtomicRAGDiscoveryAgentFactory

# discovery_manager = ManagerAgentFactory().createAgent(base_model)
# discovery_manager = SelfIterativeDiscoveryAgentFactory(num_iterations=20, graph_save_path="../data/results/causal_graph_2025-02-26_23-07-34.gml").createAgent(base_model)
discovery_manager = AtomicRAGDiscoveryAgentFactory(initial_graph=causal_graph).createAgent(base_model)

In [ ]:
print(discovery_manager.system_prompt)

In [ ]:
causal_graph = discovery_manager.run("""
            KUALA LUMPUR: Bursa Malaysia downtrend could be far from over as there is always more room to decline depending on the severity of COVID-19 pandemic and oil price war, said economists after key benchmark FBM KLCI took another beating in the early trading session yesterday. To what may be seen as continued selling pressure from last week’s Friday the 13th, the FTSE Bursa Malaysia KLCI (FBM KLCI) lost 44.99 points to 1,299.76 at 9.10am yesterday, compared with Friday’s close of 1,344.75, after opening 25.38 points lower at 1,319.37 yesterday morning. At market closing yesterday, the FBM KLCI closed at 4.77 per cent lower to 1,280.63 points with turnover at 4.473 billion shares valued at RM3.687 billion. Bank Islam chief economist Dr Mohd Afzanizam Abdul Rashid said if history is of any guide, the FBM KLCI has fallen sharply between January 11, 2008 (1,516.22 points) and October 29, 2009 (829.41 points). He said during the Asia Financial Crisis in 1997/1998, the FBM KLCI was down massively by 79.2 per cent between February 25, 1997 (1,265.01 points) and September 1, 1998 (262.7 points). "For now, the FBM KLCI touches its peak at 1,895.18 points on April 19, 2018 and has plunged by 29.0 per cent to 1,344.75 points as of March 13, 2020. There is always more room to decline obviously due to the virus outbreak and oil price war," he told NST Business. CGS-CIMB analyst Ivy Ng Lee Fang said the research firm has cut its year-end FBM KLCI target to 1,449 points. "We advise investors to seek shelter in defensive and high-dividend-yield stocks until the concerns over the global spread of COVID-19 subside. "These, together with Malaysia’s unexpected change of coalition government, could pose downside risks to corporate earnings, which are difficult to measure at this time," it said. Ng said during the global financial crisis, FBM KLCI fell by 45 per cent from its peak to 829 points, its lowest ever and FBM KLCI earnings fell by 8.7 per cent in 2009. She said there is a potential earning downside risk of 10.3 per cent to its current 1.6 per cent FBM KLCI earnings per share growth forecast if the earnings risk resembles that of during the global financial crisis decline (-8.7 per cent). Meanwhile, AxiCorp market strategist Stephen Innes said small and medium enterprises (SMEs) are most at risk given they generally operate on small operating cushions and will need to dive into credit lines if not hefty interest credit card to keep things afloat. "Unless governments actually get money physically into the hands that need it the most things can turn bad. This will put pressure on bank stocks due to credit line drawdown and an increase in defaults. "We know oil and gas and airlines are in a bad way due to travel restrictions imposed around the world. "At this stage, I see no way out and expect more investors to move into cash or keep investment portfolio very light primarily in recessionary proof areas like healthcare. "But honestly with credit under duress, I think cash is the place to be. The only endgame in sight is when the world sees COVID-19 ends," he said.
                           """)

In [ ]:
# topic = "impact of exercise on mental health"

import random
topic = random.choice(list(discovery_manager.causal_graph.nodes))

print(f"Topic: {topic}")
causal_graph = discovery_manager.run(topic)

In [ ]:
discovery_manager.tools['graph_retriever'].update_graph(causal_graph)

In [ ]:
# Plot causal graph

import networkx as nx
import matplotlib.pyplot as plt
import datetime


current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
add_labels = causal_graph.number_of_nodes() < 20
nx.draw(causal_graph, with_labels=add_labels, font_weight='bold')
nx.write_gml(causal_graph, f"../data/results/causal_graph_{current_time}.gml", stringizer=str)
plt.savefig(f"../data/results/causal_graph_{current_time}.png")

In [ ]:
from causal_world_modelling_agent.agents.causal_inference.causal_inference_agent import CausalInferenceAgentFactory

inference_agent = CausalInferenceAgentFactory().createAgent(base_model)

In [ ]:
import networkx as nx

node_data = {'COVID-19 pandemic severity': {'description': 'Severity of the COVID-19 outbreak affecting market conditions.', 'type': 'string', 'values': ['low', 'moderate', 'high']}, 'Oil price war impact': {'description': 'Impact of the oil price war on economic conditions and market sentiment.', 'type': 'string', 'values': ['low', 'moderate', 'high']}, 'Investor sentiment': {'description': 'Overall market sentiment of investors, which reacts to economic and financial uncertainties.', 'type': 'string', 'values': ['optimistic', 'neutral', 'pessimistic']}, 'FBM KLCI index level': {'description': 'The current level of the FBM KLCI stock index as observed during trading.', 'type': 'float', 'values': 'Real numbers representing index levels'}, 'Government intervention': {'description': 'Government action (or inaction) in terms of fiscal stimulus and liquidity injection.', 'type': 'string', 'values': ['active', 'inactive']}, 'Credit conditions': {'description': 'The state of credit lines and broader credit conditions affecting bank stocks and SMEs.', 'type': 'string', 'values': ['normal', 'stressed']}}
G = nx.DiGraph()
for key, value in node_data.items():
    G.add_node(key, **{"name": key, **value})

edge_data = [('COVID-19 pandemic severity', 'Investor sentiment', {'description': 'Increased COVID-19 severity deteriorates investor sentiment.', 'contextual_information': 'High outbreak risk reduces investor confidence.', 'type': 'direct', 'strength': 'strong', 'confidence': 'high', 'function': None}), ('Oil price war impact', 'Investor sentiment', {'description': 'The oil price war increases economic uncertainty, negatively affecting investor sentiment.', 'contextual_information': 'Heightened oil price conflicts add to the risk environment.', 'type': 'direct', 'strength': 'strong', 'confidence': 'high', 'function': None}), ('Investor sentiment', 'FBM KLCI index level', {'description': 'Negative investor sentiment leads to selling pressure and market decline.', 'contextual_information': 'Continued pessimism in the market has led to lower index levels.', 'type': 'direct', 'strength': 'moderate', 'confidence': 'high', 'function': None}), ('Government intervention', 'Credit conditions', {'description': 'Lack of active government intervention worsens credit conditions.', 'contextual_information': 'Insufficient fiscal measures leave credit lines stressed.', 'type': 'indirect', 'strength': 'moderate', 'confidence': 'medium', 'function': None}), ('Credit conditions', 'Investor sentiment', {'description': 'Stressed credit conditions further dampen investor sentiment.', 'contextual_information': 'Liquidity stress from credit issues heightens market fear.', 'type': 'indirect', 'strength': 'moderate', 'confidence': 'medium', 'function': None}), ('Credit conditions', 'FBM KLCI index level', {'description': 'Poor credit conditions contribute to market declines.', 'contextual_information': 'Deterioration in credit may lead to defaults and further index drops.', 'type': 'indirect', 'strength': 'moderate', 'confidence': 'medium', 'function': None})]
for source, target, attrs in edge_data:
    G.add_edge(source, target, **attrs)


nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
causal_effect, inferred_graph = inference_agent.run(
    task="Compute the causal effect of the variables in the context of the COVID-19 pandemic",
    additional_args={
            "causal_graph": G,
            "target_variable": "FBM KLCI index level",
            "observations": [
                {
                    "name": "COVID-19 pandemic severity",
                    "current_value": "low"
                }
            ]
        }
    )

In [ ]:
for key, value in inferred_graph.nodes(data=True):
    print(key)
    for k, v in value.items():
        print(f"\t{k}: {v}")